In [ ]:
import pandas as pd
import numpy as np
from heapq import  *
from math import *
from binarytree import Node
from queue import PriorityQueue

In [ ]:
data_Train = pd.read_csv('trainData.txt', sep=" ", header=None)
data_Label   = pd.read_csv('trainLabel.txt', sep=" ", header=None)

In [ ]:
word = pd.read_csv('words.txt', sep=" ", header=None)

In [ ]:
data_Train.columns = ["docid", "wordid"]
data_Train=data_Train.groupby('docid')['wordid'].apply(list)

In [ ]:
for i in range(0,1499):
    try:
        data_Train[i]
    except:
        data_Train[i] = np.nan

In [ ]:
df = pd.DataFrame(columns=["Class","words"])

In [ ]:
for i in range(0,1500):
    df = df.append({"Class":data_Label[0][i] ,"words" :data_Train[i+1]}, ignore_index=True)

In [ ]:
df = df.dropna()

In [ ]:
myList = [0] * 6969
new_df = []
for i in df.values:
    myList[0] = i[0]
    for j in i[1]:
        myList[j+1] = 1
    new_df.append(myList)
    myList = [0] * 6969

In [ ]:
df = pd.DataFrame.from_records(new_df)

In [ ]:
def average_information_gain(prev_information_gain, N1, N2):
    #print(prev_information_gain, N1, N2)
    N = N1 + N2
    if N1 == 0 and N2 == 0:
        gain = 0
    elif N1 == 0:
        gain = prev_information_gain -  (1/2 * log2(N2/N))
    elif N2 == 0:
        gain = prev_information_gain - (1/2 * log2(N1/N))
    else:
        gain = prev_information_gain - (1/2 * log2(N1/N) + 1/2 * log2(N2/N))
    return(gain)

def weighted_information_gain(prev_information_gain, N1, N2):
    N = N1 + N2
    if N1 == 0 and N2 == 0:
        gain = 0
    elif N1 == 0:
        gain = prev_information_gain - (N2/N * log2(N2/N))
    elif N2 == 0:
        gain = prev_information_gain - (N1/N * log2(N1/N))
    else:
        gain = prev_information_gain - (N1/N * log2(N1/N) + N2/N * log2(N2/N))
    return(gain)

In [ ]:
def find_max_gain(df, prev_information_gain, avg = True):
    
    N = 6969
    N1 = len(df[(df[0] == 1)])
    N2 = len(df[(df[0] == 2)])
    
    gains = -inf
    temp = 0
    
    # AVERAGE = TRUE
    
    if avg:
        for i in range(1, N):
            N1_1 = len(df[(df[0] == 1) & (df[i] == 1)])
            N1_0 = len(df[(df[0] == 2) & (df[i] == 1)])

            #N2_1 = len(df[(df[0] == 1) & (df[i] == 0)])
            #N2_0 = len(df[(df[0] == 2) & (df[i] == 0)])

            N2_1 = N1 - N1_1
            N2_0 = N2 - N1_0


            entropy1 = average_information_gain(0, N1_1,N1_0)
            entropy2 = average_information_gain(0, N2_1,N2_0)
            #gain1 = average_information_gain(prev_information_gain, N2_1,N2_0)

            total = N1_1 + N2_1 + N1_0 + N2_0
            try:
                gain = -1/2 * entropy1 - 1/2 * entropy2
            except:
                gain = 0


            #print(gain)

            #print(N1_1, N1_0, N2_1, N2_0)

            if gains < gain:
                gains = gain
                temp = i
                
                
        # AVERAGE = FALSE
        
        else:
            for i in range(1, N):
            N1_1 = len(df[(df[0] == 1) & (df[i] == 1)])
            N1_0 = len(df[(df[0] == 2) & (df[i] == 1)])

            #N2_1 = len(df[(df[0] == 1) & (df[i] == 0)])
            #N2_0 = len(df[(df[0] == 2) & (df[i] == 0)])

            N2_1 = N1 - N1_1
            N2_0 = N2 - N1_0


            entropy1 = weighted_information_gain(0, N1_1,N1_0)
            entropy2 = weighted_information_gain(0, N2_1,N2_0)
            #gain1 = average_information_gain(prev_information_gain, N2_1,N2_0)

            total = N1_1 + N2_1 + N1_0 + N2_0
            try:
                gain = -(N1_1 + N1_0)/total * entropy1 - (N2_1 + N2_0)/total * entropy2
            except:
                gain = 0


            #print(gain)

            #print(N1_1, N1_0, N2_1, N2_0)

            if gains < gain:
                gains = gain
                temp = i
            
    
    #heappush(h, (-gains, temp, Node))
    #h.append([gains, temp, Node])
    
    return [gains, temp]
        

                

In [ ]:
class treeNode:
    
    def __init__(self, data, condition = -1, gain = -inf):
        self.data = data
        self.left = None
        self.right = None
        self.condition = condition
        self.gain = gain
    
    def addLeft(self, element):
        self.left = element
    
    def addRight(self, element):
        self.right = element
        
    def addCondition(self, cond):
        self.condition = cond
        
        
    def calcGain(self, prev_information_gain, avg = True):
        temp = find_max_gain(self.data, prev_information_gain, avg)
        self.gain = temp[0]
        self.condition = temp[1]
        
    
        

In [ ]:
## Calculation Average Tree
def createAvgTree(NumOfNodes):
    h = []
    df = pd.DataFrame.from_records(new_df)
    #df = pd.DataFrame.from_records(new_df)
    prev_information_gain = 0
    N1 = len(df[(df[0] == 1)])
    N2 = len(df[(df[0] == 2)])

    prev_information_gain = average_information_gain(0, N1, N2)
    Node = treeNode(data = df)
    Node.calcGain(prev_information_gain)
    unused = []
    unused.append(Node)

    #Node.data.head()

    AvgRoot = Node

    heappush(h, (-1 * Node.gain, Node.condition, Node))

    #print(Node.gain, Node.condition)
    for i in range(NumOfNodes):
        #split(Node)
        #-----------------
        maximum = heappop(h)
        print(maximum)
        print(unused)
        print(h)
        for j in unused:
            if j == maximum[2]:
                print("hey")
                Node = j
                unused.remove(j)
                break


        df = Node.data
        prev_information_gain = Node.gain

        left = treeNode(data = df[(df[Node.condition] == 0)])
        right = treeNode(data = df[(df[Node.condition] == 1)])

        left.calcGain(prev_information_gain)
        right.calcGain(prev_information_gain)

        heappush(h, (-1 * left.gain, left.condition, left))
        heappush(h, (-1 * right.gain, right.condition, right))

        Node.addLeft(left)
        Node.addRight(right)

        unused.append(left)
        unused.append(right)
        #----------------------

        print(Node.gain, Node.condition)

In [ ]:
createAvgTree(10)

In [ ]:
## Calculating Weighted Tree

def createWeightedTree(NumOfNodes):
    h = []
    df = pd.DataFrame.from_records(new_df)
    #df = pd.DataFrame.from_records(new_df)
    prev_information_gain = 0
    N1 = len(df[(df[0] == 1)])
    N2 = len(df[(df[0] == 2)])

    prev_information_gain = weighted_information_gain(0, N1, N2)
    Node = treeNode(data = df)
    Node.calcGain(prev_information_gain)
    unused = []
    unused.append(Node)

    #Node.data.head()

    WeightedRoot = Node

    heappush(h, (-1 * Node.gain, Node.condition, Node))

    #print(Node.gain, Node.condition)
    for i in range(NumOfNodes):
        #split(Node)
        #-----------------
        maximum = heappop(h)
        print(maximum)
        print(unused)
        print(h)
        for j in unused:
            if j == maximum[2]:
                print("hey")
                Node = j
                unused.remove(j)
                break


        df = Node.data
        prev_information_gain = Node.gain

        left = treeNode(data = df[(df[Node.condition] == 0)])
        right = treeNode(data = df[(df[Node.condition] == 1)])

        left.calcGain(prev_information_gain, False)
        right.calcGain(prev_information_gain, False)

        heappush(h, (-1 * left.gain, left.condition, left))
        heappush(h, (-1 * right.gain, right.condition, right))

        Node.addLeft(left)
        Node.addRight(right)

        unused.append(left)
        unused.append(right)
        #----------------------

        print(Node.gain, Node.condition)

In [ ]:
createWeightedTree(10)

In [ ]:
AvgTraversal = []
WeightedTraversal = []
def inorderTraversal(root):
        if root:
            res = inorderTraversal(root.left) 
            print(root.condition)
            res = res + inorderTraversal(root.right)


In [ ]:
AvgTraversal = inorderTraversal(AvgRoot)
WeightedTraversal = inorderTraversal(WeightedRoot)

In [ ]:
#Node = root
#pred = []
def classify(data, pred):
    if Node.condition == inf:
        df = Node.data
        len1 = len(df[df[0] == 1])
        len2 = len(df[df[0] == 2])
        
        if len1 > len2:
            pred.append(1)
        else:
            pred.append(2)
    elif data[Node.condition] == 1:
        Node = Node.right
    else:
        Node = Node.left
    
            

In [ ]:
data_Test = pd.read_csv('testData.txt', sep=" ", header=None)
data_Label   = pd.read_csv('testLabel.txt', sep=" ", header=None)

word = pd.read_csv('words.txt', sep=" ", header=None)

data_Test.columns = ["docid", "wordid"]
data_Test=data_Test.groupby('docid')['wordid'].apply(list)

for i in range(0,1499):
    try:
        data_Test[i]
    except:
        data_Test[i] = np.nan
        
df = pd.DataFrame(columns=["Class","words"])

for i in range(0,1500):
    df = df.append({"Class":data_Label[0][i] ,"words" :data_Test[i+1]}, ignore_index=True)
    
df = df.dropna()

myList = [0] * 6969
new_df = []
for i in df.values:
    myList[0] = i[0]
    for j in i[1]:
        myList[j+1] = 1
    new_df.append(myList)
    myList = [0] * 6969
    
    
test = pd.DataFrame.from_records(new_df)

In [ ]:
train

In [ ]:
test0 = test[0]
del test[0]

In [ ]:
AvgPred = []
WeightedPred = []
Node = AvgRoot
for data in train.values:
    classify(data, AvgPred)

Node = WeightedRoot
for data in train.values:
    classify(data, WeightedPred)


In [ ]:
tot = len(pred)
AvgCorrect = 0
for i,j in zip(pred, test0):
    if i == j:
        AvgCorrect += 1
        
WeightedCorrect = 0
for i,j in zip(pred, test0):
    if i == j:
        WeightedCorrect += 1

In [ ]:
AvgAccuracy = AvgCorrect/tot * 100
#For 10 Nodes

In [ ]:
WeightedAccuracy = WeightedCorrect/tot * 100
#For 10 Nodes

In [ ]:
testAvgAccuracy = []
trainWeightedAccuracy = []

testWeightedAccuracy = []
trainAvgAccuracy = []

for i in range(1,101):
    createAvgTree(ii)
    createWeightedTree(i)
    
    AvgPred = []
    WeightedPred = []
    Node = AvgRoot
    for data in test.values:
        classify(data, AvgPred)

    Node = WeightedRoot
    for data in test.values:
        classify(data, WeightedPred)

    tot = len(pred)
    AvgCorrect = 0
    for i,j in zip(pred, test0):
        if i == j:
            AvgCorrect += 1

    WeightedCorrect = 0
    for i,j in zip(pred, test0):
        if i == j:
            WeightedCorrect += 1
    
    AvgAccuracy = AvgCorrect/tot * 100

    WeightedAccuracy = WeightedCorrect/tot * 100
    
    
    testAvgAccuracy.append(AvgAccuracy)
    testWeightedAccuracy.append(WeightedAccuracy)
    
    
    AvgPred = []
    WeightedPred = []
    Node = AvgRoot
    for data in train.values:
        classify(data, AvgPred)

    Node = WeightedRoot
    for data in train.values:
        classify(data, WeightedPred)

    tot = len(pred)
    AvgCorrect = 0
    for i,j in zip(pred, test0):
        if i == j:
            AvgCorrect += 1

    WeightedCorrect = 0
    for i,j in zip(pred, test0):
        if i == j:
            WeightedCorrect += 1
    
    AvgAccuracy = AvgCorrect/tot * 100

    WeightedAccuracy = WeightedCorrect/tot * 100
    
    trainAvgAccuracy.append(AvgAccuracy)
    trainWeightedAccuracy.append(WeightedAccuracy)
    

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(range(1,101), testAvgAccuracy)
plt.plot(range(1,101), trainAvgAccuracy)
plt.show()

In [ ]:
plt.plot(range(1,101), testWeightedAccuracy)
plt.plot(range(1,101), trainWeightedAccuracy)
plt.show()